# Transfer learning with PyTorch
We're going to train a neural network to classify dogs and cats.

## Init, helpers, utils, ...

In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

In [3]:
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
from IPython.core.debugger import set_trace

In [4]:
# %load my_train_helper.py
def get_trainable(model_params):
    return (p for p in model_params if p.requires_grad)


def get_frozen(model_params):
    return (p for p in model_params if not p.requires_grad)


def all_trainable(model_params):
    return all(p.requires_grad for p in model_params)


def all_frozen(model_params):
    return all(not p.requires_grad for p in model_params)


def freeze_all(model_params):
    for param in model_params:
        param.requires_grad = False


# The Data - DogsCatsDataset

## Transforms

In [5]:
from torchvision import transforms

_image_size = 224
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]


train_trans = transforms.Compose([
    transforms.Resize(256),  # some images are pretty small
    transforms.RandomCrop(_image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3, .3, .3),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])
val_trans = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(_image_size),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])

## Dataset

The implementation of the dataset does not really.

In [6]:
from torchvision.datasets.folder import ImageFolder

In [8]:
train_ds = ImageFolder("dogscats/training_set/", transform=train_trans)
val_ds = ImageFolder("dogscats/test_set/", transform=val_trans)

batch_size = 32
n_classes = 2

Use the following if you want to use the full dataset:

In [9]:
# train_ds = DogsCatsDataset("../data/raw", "train", transform=train_trans)
# val_ds = DogsCatsDataset("../data/raw", "valid", transform=val_trans)

# batch_size = 128
# n_classes = 2

In [10]:
len(train_ds), len(val_ds)

(8000, 2000)

## DataLoader
Batch loading for datasets with multi-processing and different sample strategies.

In [11]:
from torch.utils.data import DataLoader


train_dl = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=16,
)

val_dl = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=16,
)

# The Model
PyTorch offers quite a few [pre-trained networks](https://pytorch.org/docs/stable/torchvision/models.html) such as:
- AlexNet
- VGG
- ResNet
- SqueezeNet
- DenseNet
- Inception v3

And there are more available via [pretrained-models.pytorch](https://github.com/Cadene/pretrained-models.pytorch):
- NASNet,
- ResNeXt,
- InceptionV4,
- InceptionResnetV2, 
- Xception, 
- DPN,
- ...

We'll use a simple resnet18 model:

In [14]:
from torchvision import models

model = models.resnet18(pretrained=True)

In [15]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
import torchsummary

torchsummary.summary(model, (3, 224, 224), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [17]:
nn.Linear(2, 1, bias=True)

Linear(in_features=2, out_features=1, bias=True)

In [18]:
# Freeze all parameters manually
for param in model.parameters():
    param.requires_grad = False

In [19]:
# Or use our convenient functions from before
freeze_all(model.parameters())
assert all_frozen(model.parameters())

Replace the last layer with a linear layer. New layers have `requires_grad = True`.

In [20]:
model.fc = nn.Linear(512, n_classes)

In [21]:
assert not all_frozen(model.parameters())

In [23]:
def get_model(n_classes=2):
    model = models.resnet18(pretrained=True)
    freeze_all(model.parameters())
    model.fc = nn.Linear(512, n_classes)
    model = model.to(DEVICE)
    return model


model = get_model()

# The Loss

In [25]:
criterion = nn.CrossEntropyLoss()

# The Optimizer

In [26]:
optimizer = torch.optim.Adam(
    get_trainable(model.parameters()),
    lr=0.001,
    # momentum=0.9,
)

# The Train Loop

In [27]:
N_EPOCHS = 1

for epoch in range(N_EPOCHS):
    
    # Train
    model.train()  # IMPORTANT
    
    total_loss, n_correct, n_samples = 0.0, 0, 0
    for batch_i, (X, y) in enumerate(train_dl):
        X, y = X.to(DEVICE), y.to(DEVICE)
        
        optimizer.zero_grad()
        y_ = model(X)
        loss = criterion(y_, y)
        loss.backward()
        optimizer.step()
        
        # Statistics
        print(
            f"Epoch {epoch+1}/{N_EPOCHS} |"
            f"  batch: {batch_i} |"
            f"  batch loss:   {loss.item():0.3f}"
        )
        _, y_label_ = torch.max(y_, 1)
        n_correct += (y_label_ == y).sum().item()
        total_loss += loss.item() * X.shape[0]
        n_samples += X.shape[0]
    
    print(
        f"Epoch {epoch+1}/{N_EPOCHS} |"
        f"  train loss: {total_loss / n_samples:9.3f} |"
        f"  train acc:  {n_correct / n_samples * 100:9.3f}%"
    )
    
    
    # Eval
    model.eval()  # IMPORTANT
    
    total_loss, n_correct, n_samples = 0.0, 0, 0
    with torch.no_grad():  # IMPORTANT
        for X, y in val_dl:
            X, y = X.to(DEVICE), y.to(DEVICE)
                    
            y_ = model(X)
        
            # Statistics
            _, y_label_ = torch.max(y_, 1)
            n_correct += (y_label_ == y).sum().item()
            loss = criterion(y_, y)
            total_loss += loss.item() * X.shape[0]
            n_samples += X.shape[0]

    
    print(
        f"Epoch {epoch+1}/{N_EPOCHS} |"
        f"  valid loss: {total_loss / n_samples:9.3f} |"
        f"  valid acc:  {n_correct / n_samples * 100:9.3f}%"
    )

Epoch 1/1 |  batch: 0 |  batch loss:   0.775
Epoch 1/1 |  batch: 1 |  batch loss:   0.611
Epoch 1/1 |  batch: 2 |  batch loss:   0.690
Epoch 1/1 |  batch: 3 |  batch loss:   0.715
Epoch 1/1 |  batch: 4 |  batch loss:   0.686
Epoch 1/1 |  batch: 5 |  batch loss:   0.463
Epoch 1/1 |  batch: 6 |  batch loss:   0.467
Epoch 1/1 |  batch: 7 |  batch loss:   0.484
Epoch 1/1 |  batch: 8 |  batch loss:   0.384
Epoch 1/1 |  batch: 9 |  batch loss:   0.395
Epoch 1/1 |  batch: 10 |  batch loss:   0.316
Epoch 1/1 |  batch: 11 |  batch loss:   0.403
Epoch 1/1 |  batch: 12 |  batch loss:   0.330
Epoch 1/1 |  batch: 13 |  batch loss:   0.420
Epoch 1/1 |  batch: 14 |  batch loss:   0.312
Epoch 1/1 |  batch: 15 |  batch loss:   0.369
Epoch 1/1 |  batch: 16 |  batch loss:   0.264
Epoch 1/1 |  batch: 17 |  batch loss:   0.298
Epoch 1/1 |  batch: 18 |  batch loss:   0.299
Epoch 1/1 |  batch: 19 |  batch loss:   0.391
Epoch 1/1 |  batch: 20 |  batch loss:   0.271
Epoch 1/1 |  batch: 21 |  batch loss:   0.32

Epoch 1/1 |  batch: 178 |  batch loss:   0.152
Epoch 1/1 |  batch: 179 |  batch loss:   0.044
Epoch 1/1 |  batch: 180 |  batch loss:   0.098
Epoch 1/1 |  batch: 181 |  batch loss:   0.102
Epoch 1/1 |  batch: 182 |  batch loss:   0.131
Epoch 1/1 |  batch: 183 |  batch loss:   0.101
Epoch 1/1 |  batch: 184 |  batch loss:   0.106
Epoch 1/1 |  batch: 185 |  batch loss:   0.069
Epoch 1/1 |  batch: 186 |  batch loss:   0.082
Epoch 1/1 |  batch: 187 |  batch loss:   0.041
Epoch 1/1 |  batch: 188 |  batch loss:   0.104
Epoch 1/1 |  batch: 189 |  batch loss:   0.165
Epoch 1/1 |  batch: 190 |  batch loss:   0.092
Epoch 1/1 |  batch: 191 |  batch loss:   0.159
Epoch 1/1 |  batch: 192 |  batch loss:   0.186
Epoch 1/1 |  batch: 193 |  batch loss:   0.141
Epoch 1/1 |  batch: 194 |  batch loss:   0.153
Epoch 1/1 |  batch: 195 |  batch loss:   0.154
Epoch 1/1 |  batch: 196 |  batch loss:   0.067
Epoch 1/1 |  batch: 197 |  batch loss:   0.209
Epoch 1/1 |  batch: 198 |  batch loss:   0.245
Epoch 1/1 |  

# Exercise
- Create your own module which takes any of the existing pre-trained model as backbone and adds a problem specific head.

In [ ]:
class Net(nn.Module):
    def __init__(self, backbone: nn.Module, n_classes: int):
        super().__init__()
        # self.backbone
        # self.head = init_head(n_classes)
        
    def forward(self, x):
        # TODO
        return x